In [28]:
import pandas as pd
import numpy as np

# Load data
data = pd.read_csv("Project-II/Fraud.csv")
#print(data.head())

In [29]:
size,dim = data.shape
print(f"There are {size} online payments and {dim} variables.")
data = data.drop('isFlaggedFraud', axis=1)
print(data.head())

There are 6362620 online payments and 11 variables.
   step      type    amount  ... oldbalanceDest  newbalanceDest  isFraud
0     1   PAYMENT   9839.64  ...            0.0             0.0        0
1     1   PAYMENT   1864.28  ...            0.0             0.0        0
2     1  TRANSFER    181.00  ...            0.0             0.0        1
3     1  CASH_OUT    181.00  ...        21182.0             0.0        1
4     1   PAYMENT  11668.14  ...            0.0             0.0        0

[5 rows x 10 columns]


Check for missing values

In [30]:
print(data.isnull().sum())

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
dtype: int64


In [ ]:
# Exploring transaction type
#print(data.type.value_counts())

In [31]:
type = data["type"].value_counts()
transactions = type.index
quantity = type.values

import plotly.express as px
figure = px.pie(data,
             values=quantity,
             names=transactions,hole = 0.5,
             title="Distribution of Transaction Type")
figure.show()

In [32]:
data["type"] = data["type"].map({"CASH_OUT": 1, "PAYMENT": 2,
                                 "CASH_IN": 3, "TRANSFER": 4,
                                 "DEBIT": 5})
#data["isFraud"] = data["isFraud"].map({0: "No Fraud", 1: "Fraud"})
#print(data.head())

In [33]:
# Checking correlation
correlation = data.iloc[:,[0,1,2,4,5,7,8,9]].corr()
print(correlation["isFraud"].sort_values(ascending=False))

isFraud           1.000000
amount            0.076688
step              0.031578
type              0.016171
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64


In [ ]:
#data["isFraud"] = data["isFraud"].map({0: "No", 1: "Yes"})

### Oline payments Fraud Dtection models

In [34]:
from sklearn.model_selection import train_test_split

# splitting the data
X = np.array(data[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]])
y = np.array(data[["isFraud"]])

# training a decision tree
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

### Grid Search

In [ ]:
# GRID SEARCH
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [2, 3, 4]
}
grid_search = GridSearchCV(
estimator = GradientBoostingClassifier(random_state=42),
param_grid = param_grid,
scoring = 'recall', 
cv = 3)

grid_search.fit(X_train, y_train)

print("Best parameters found: ". grid_search.best_params_)
print("Best Recall: ", grid_search.best_score_)

### Metrics Visualization


In [ ]:
# Visualizacion de resultados
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, auc
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, adjusted_rand_score

y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]


report = classification_report(y_test,y_pred)
print(report)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC: {roc_auc}")

precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall, precision)
print(f"Precision-Recall Auc: {pr_auc}")

### Decision trees

In [ ]:
#DECISION TREE CLASSIFIER
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)




### Random Forest

In [ ]:
# RANDOM FOREST CLASSIFIER

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
n_estimators = 100,
class_weight='balanced',
random_state = 42)

model.fit(X_train, y_train)

'''
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    635445
           1       0.92      0.89      0.91       817

    accuracy                           1.00    636262
   macro avg       0.96      0.95      0.95    636262
weighted avg       1.00      1.00      1.00    636262

ROC-AUC: 0.9956702544240986
Precision-Recall Auc: 0.9659164838130454

'''
#Con class_weight = 'balaanced' :
'''
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    635445
           1       0.92      0.89      0.90       817

    accuracy                           1.00    636262
   macro avg       0.96      0.94      0.95    636262
weighted avg       1.00      1.00      1.00    636262

ROC-AUC: 0.9968962266855792
Precision-Recall Auc: 0.9668656031185896
'''

### Gradient Boosting

In [35]:
# GRADIENT BOOSTING CLASSIFIER

from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(
n_estimators=100, 
random_state=42)

model.fit(X_train, y_train)


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



### XGBoost

In [ ]:
# MODELO XGBoost

from xgboost import XGBClassifier

model = XGBClassifier (
n_estimators = 300,
learning_rate = 0.1,
class_weight='balanced',
random_state = 42)

model.fit(X_train, y_train)

'''
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    635445
           1       0.90      0.78      0.83       817

    accuracy                           1.00    636262
   macro avg       0.95      0.89      0.92    636262
weighted avg       1.00      1.00      1.00    636262

ROC-AUC: 0.9996006326121193
Precision-Recall Auc: 0.920761258323663 
'''

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create a LogisticRegression model
clf = LogisticRegression(random_state=42, max_iter=1000)

# they require scaling

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))

### Support Vector Machines

In [ ]:
from sklearn.svm import SVC

# Create a SVM model
clf = SVC(random_state=42)

# they require scaling

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Display results
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Sensitivity (Recall):", recall_score(y_test, y_pred, pos_label='Yes'))